## Efficiently measurement of Tsallis Stabilizer Entropy with quantum computers

Python code to efficiently measure stabilizer entropy (SE) using two copies of a state in the Bell basis. Simulates various states with optional depolarizing noise and finite number of measurements. Runs on qiskit.

Companion Code for "Efficient stabilizer entropies for quantum computers" by T. Haug, S. Lee and  M.S. Kim.

@Tobias Haug, Imperial College London
thaug@ic.ac.uk

@Soovin Lee, Imperial College London
soovinlee310@gmail.com


This code measures Tsallis SE using two copies of a state in the Bell basis.
It implements three algorithms:
1. Measure SE via Bell measurements (efficient for odd n)
2. Measure SE via Bell measurements on complex conjugate, followed by Pauli measurements (efficient for integer n>1)
3. Gradients of SES via Bell measurements (efficient for odd n)

As comparison, also computes exact SE using statevector simulator.
Also can simulate depolarizing error and correct it via error mitigation.



In [103]:
import qiskit as qk
import numpy as np
from qiskit.quantum_info import PauliList, Pauli
from qiskit.opflow.state_fns import StateFn
from qiskit.opflow import X, Y, Z, I
from qiskit.circuit.library import p, u



In [104]:
##parameters to be adjusted

n_qubits=3 #number qubits
depth=20 #number of layers of parameterized single qubit rotations, (twice of d as defined in manuscript)

type_circuit=2##0: T state, 1: R state, 2: circuit with y,z rotations and CNOT in nearest neighbor chain configuration, 3: + state

type_parameters=0 ##0: random parameters, 1: random clifford parameters \in\{0,\pi/2,\pi,3\pi/2} + n_nonclifford parameters which represent T-gates, 2: random clifford parameter with initial layer of n_nonclifford magic states
n_nonclifford=1 #number of equivalent T-gates for type_parameters=1 or number of magic states for type_parameters=2

depolarization_factor=0 ##global depolariziation error (not implemented for conjugate method and gradient)

n=3 #entropic index of Tsallis-n SE

random_seed=1 #seed of random generator

n_samples=10000 ##number of measurements for Bell measurement and gradient computation
n_samples_cc=1000 ##number of measurements of complex conjugate method (is slower)


n_resample=10*n_samples ##randomly re-draw Bell measurements to increase accuracy

gradient_index=0# compute gradient for which parameter index, -1 to not compute gradient



In [105]:
def numberToBase(n, b,n_qubits):
    ##get all possible bases
    if n == 0:
        return np.zeros(n_qubits,dtype=int)
    digits = np.zeros(n_qubits,dtype=int)
    counter=0
    while n:
        digits[counter]=int(n % b)
        n //= b
        counter+=1
    return digits[::-1]

def integer_to_pauli(pauli):
    ##produces a Pauli String 
    string=[]
    for i in (range(len(pauli))):
        if(pauli[i]==0):
            x="I"
        elif(pauli[i]==1):
            x="X"
        elif(pauli[i]==2):
            x="Z" 
        elif(pauli[i]==3):
            x="Y"
        string.append(x)
    return "".join(string)

def SWAP_purity(bitstrings_sampled,sampled_states_probs):
    """
    get purity from bitstrings and probabilities of Bell measurement
    """
    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2

    ##do AND between each pair of bits from first and second copy, then check parity
    parity_list=np.array([np.sum((bitstrings_sampled[i,:n_qubits]+bitstrings_sampled[i,n_qubits:])>1)%2 for i in range(len(bitstrings_sampled))],dtype=int)

    ##purity is 1-2*(probability of odd parity)
    purity=1-2*np.sum(sampled_states_probs*parity_list)
    return purity

def pauli_measurement(circuit_in, pauli, qr, cr, barrier=True):
    """
    Add the proper post-rotation gate on the circuit to measure a given Pauli.
    Assumes that input circuit has NO measurement at end, but a classical register is there

    Args:
        circuit (QuantumCircuit): the circuit to be modified.
        pauli (Pauli): the pauli will be added.
        qr (QuantumRegister): the quantum register associated with the circuit.
        cr (ClassicalRegister): the classical register associated with the circuit.
        barrier (bool, optional): whether or not add barrier before measurement.

    Returns:
        QuantumCircuit: the original circuit object with post-rotation gate
    """
    
    ##copy circuit so that input circuit is unchanged
    circuit=circuit_in.copy()

    

    num_qubits = pauli.num_qubits
    for qubit_idx in range(num_qubits):
        #print(qr[qubit_idx])
        if pauli.x[qubit_idx]:
            if pauli.z[qubit_idx]:
                # Measure Y
                circuit.p(-np.pi / 2, qr[qubit_idx])  # sdg
                circuit.u(np.pi/2, 0.0, np.pi, qr[qubit_idx])  # h
            else:
                # Measure X
                circuit.u(np.pi/2, 0.0, np.pi, qr[qubit_idx])  # h
        if barrier:
            circuit.barrier(qr[qubit_idx])
        circuit.measure(qr[qubit_idx], cr[qubit_idx])

    return circuit

In [106]:
def se_exact(n, n_qubits, psi, backend):
    """
    Get exact SE for infinite number of samples. 
    n: entropic index
    n_qubtis: number of qubtis
    psi: statevector
    backend: qiskit backend (statevector_simulator recommended)
    
    returns exact TsallisEntropy
    """

    #get all combinations of pauli string
    int_pauli = [numberToBase(k,4,n_qubits) for k in range(4**n_qubits)]
    string_pauli = [integer_to_pauli(int_pauli[k]) for k in range(4**n_qubits)]
    
    expval_list = []
    
    #get expectation value for each pauli string
    for op in PauliList(string_pauli):
        expval_list.append(psi.expectation_value(op))
    
    #exact calculation of SE
    expval_list = np.array(expval_list)
    TsallisEntropy = (1-np.sum(expval_list**(2*n)/(2**n_qubits)))/(n-1)
    
    return TsallisEntropy

def se_sample(n, bitstrings_sampled,sampled_states_probs, n_samples=0, n_resample=1):
    """
    Estimate SE from samples of Bell measurements. Corresponds to Algorithm 1 of paper
    Efficient for odd n
    
    
    Inputs are 
    n: index of SE, n>1
    bitstrings_sampled: j=1,...,K sampled bitstrings from K Bell measurement, first copy on first n_qubit bits, second copy on last n_qubit bits
    sampled_states_probs: respective probabilities
    n_samples: total number of measurements
    n_resample: How often to re-sample Bell measurements, good choice is 10*n_samples
    
    """
    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2

    if(n_samples==0):
        raise NameError("Not implemented")
    
    if(n<=0):
        raise NameError("n must be positive")
    
    ##reconstruct counts to verifify that n_samples is indeed correct
    counts=np.array(np.round(sampled_states_probs*n_samples),dtype=int)
    if(np.sum(counts)!=n_samples):
        raise NameError("n_samples did not match acutal counts")
    
    ##whether to sample with or without replacement
    if(n_samples<n):
        replace=True
    else:
        replace=False
    
    #check if index is odd
    if n%2 == 1:
        odd = True
    else:
        odd = False
    
    ##sample n random bitstrings from all samples without replacement, do this n_resample times
    exp_sum = 0
    for rep in range(n_resample):
        ##draw random set of bell measurement outcomes
        sample_list= rng.choice(bitstrings_sampled, n, replace=False)
        
        result = 1
        for j in range(n_qubits):
            x = sample_list[:,[j, j+n_qubits]].T #first copy: first n_qubit bits (j), second copy: last n_qubit bits (j+n_qubits)
            
            ##do parity check to compute SE
            zicheck = sum(x[0])
            izcheck = sum(x[1])
            
            if odd == True:
                if zicheck%2 == 1 and izcheck%2 == 1: #both odd 
                    result *= -1 #1/2((IxI)^n+(ZxI)^n+(IxZ)^n-(ZxZ)^n), where x is tensor product and ^ is tensor power
            else:
                if zicheck%2 == 0 and izcheck%2 == 0: #both even
                    result *= 2
                else:
                    result = 0
        exp_sum += result
    
    A = exp_sum/n_resample #returning expectation value of A
    SE = -(1-A)/(1-n)
        
    return SE


def A_sample_gradient(n, bitstrings_sampled,sampled_states_probs, bitstrings_shifted_sampled,sampled_states_shifted_probs, n_samples=0, n_resample=1):
    global sample_unshifted_list,sample_shifted_list
    """
    Estimate A_n from samples of Bell measurements.
    Efficient for odd n

    Inputs are 
    n: index of SE, n>1
    bitstrings_sampled: j=1,...,K sampled bitstrings from K Bell measurement, first copy on first n_qubit bits, second copy on last n_qubit bits
    sampled_states_probs: respective probabilities

    bitstrings_shifted_sampled: j=1,...,K sampled bitstrings shifted from K Bell measurement, first copy on first n_qubit bits, second copy on last n_qubit bits
    sampled_states_shifted_probs: respective probabilities
    
    n_samples: total number of measurements
    n_resample: How often to repeat magic estimation protocol, good choice is 10*n_samples
    
    returns A_n, which is used to compute Tsallis SE
    
    """
    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2

    if(n_samples==0):
        raise NameError("Not implemented")
    
    if(n<=1):
        raise NameError("n must be greater 1")
    
    ##reconstruct counts to verifify that n_samples is indeed correct
    counts=np.array(np.round(sampled_states_probs*n_samples),dtype=int)
    if(np.sum(counts)!=n_samples):
        raise NameError("n_samples did not match acutal counts")
    
    ##whether to sample with or without replacement
    if(n_samples<n):
        replace=True
    else:
        replace=False
    
    #check if index is odd
    if n%2 == 1:
        odd = True
    else:
        odd = False
    
    ##sample n random bitstrings from all samples without replacement, do this n_resample times
    exp_sum = 0
    for rep in range(n_resample):
        ##unshifted Bell measurements
        sample_unshifted_list= rng.choice(bitstrings_sampled, n-1, replace=False)
        
        ##Bell measurements where one state was shifted by +pi/2 or -pi/2
        sample_shifted_list= rng.choice(bitstrings_shifted_sampled, 1, replace=False)
        
        sample_list=np.array(list(sample_shifted_list)+list(sample_unshifted_list))
        
        result = 1
        for j in range(n_qubits):
            x = sample_list[:,[j, j+n_qubits]].T #first copy: first n_qubit bits (j), second copy: last n_qubit bits (j+n_qubits)
            
            zicheck = sum(x[0])
            izcheck = sum(x[1])
            
            if odd == True:
                if zicheck%2 == 1 and izcheck%2 == 1: #both odd 
                    result *= -1 #1/2((IxI)^n+(ZxI)^n+(IxZ)^n-(ZxZ)^n), where x is tensor product and ^ is tensor power
            else:
                if zicheck%2 == 0 and izcheck%2 == 0: #both even
                    result *= 2
                else:
                    result = 0
        exp_sum += result
    
    A = exp_sum/n_resample #returning expectation value of A

        
    return A

def se_sample_conjugate(n, circuit_parameters, rotation_gates, entangling_gate_index_list, n_samples=0):
    
    """
    Compute Tsallis SE using Bell measurements on complex conjugate of state.
    
    Sampled Bell measurement outcomes correspond to sampled Pauli strings.
    
    Then measure 2n-2 power of expectation value of Pauli string
    
    From this, one can estimate Tsallis SE for any n>1 efficiently.
    
    Corresponds to Algorithm 2 of paper.
    
    Input:
    index n
    circuit_parameters: parameters of circuit
    rotation_gates: Description of circuit rotations
    entangling_gate_index_list: Description of entangling gates used in circuit
    
    n_samples: number of measurements
    
    """
    
    backend = qk.Aer.get_backend('qasm_simulator')
    
    qc=get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,conjugate=True,get_statevector=False)
    job_qiskit = qk.execute(qc,backend=backend,shots=n_samples)
    job_qiskit_result=job_qiskit.result()
    
    bitstrings_sampled,sampled_states_probs = get_bitstrings(job_qiskit_result,depolarization_factor=0)
    
    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2

    if(n_samples==0):
        raise NameError("Not implemented")
    
    if(n<=1):
        raise NameError("n must be greater 1")
    

    ##base circuit, we add Pauli measurmeents later
    base_circuit, qr, cr = get_qiskit_circuit_single(circuit_parameters,rotation_gates,entangling_gate_index_list, get_statevector=False, registers=True,add_measurements=False)
    
    #print(qr[0])
             
    #print(base_circuit)
    
    ##sample n random bitstrings from all samples without replacement, do this n_resample times
    A = 0
    for rep in range(n_samples):

        ##get basis in Pauli basis instead of bitbasis
        ##map bitstrings to pauli strings
        sampled_basis = bitstrings_sampled[rep,:n_qubits]+2*bitstrings_sampled[rep,n_qubits:] 
        pauli_string = integer_to_pauli(sampled_basis) ##maps #0=I, 1=X, 2=Z, 3=Y
        #print(sampled_basis)
        #print(pauli_string)
        

        
        pauli_power=2*n-2 ##we compute the pauli to the power of 2n-2 
        ##we do this by taking 2n-2 shots, then compute the expectation over those 2n-2 shots

        #measure state in above eigenbasis
        single_state_circuit= pauli_measurement(base_circuit, Pauli(pauli_string), qr, cr, barrier=True)
        #print(single_state_circuit)
        job_qiskit = qk.execute(single_state_circuit,backend=backend,shots=pauli_power)
        job_qiskit_result =job_qiskit.result()
        bitstring, probability = get_bitstrings(job_qiskit_result,depolarization_factor=0)
        
        #print(bitstring)
        result=1
        for i in range(pauli_power):
            added_bitstring=0
            for j in range(n_qubits):
                if(pauli_string[j]!="I"): ##not identity Pauli
                    added_bitstring+=bitstring[i][j]
                    
            ##check whether i-th outcome has even or odd parity, then assign value of measured pauli eigenvalue
            if added_bitstring%2==1:
                expval_full = -1
            else:
                expval_full = 1

            result *= expval_full
            ##multiply together to get the estimation for the power
        #print(result)
            
        A += result
    
    A = A/n_samples #returning expectation value of A
    SE = -(1-A)/(1-n)
    
    return SE


In [107]:
def get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,gradient_index=-1,sign_shift=1,conjugate=False,get_statevector=False):
    """
    creates qiskit circuit for Bell measurements needed for measuring Tsallis SE. 
    Prepares two copies of state of various types, then Bell transformation, then sampling in computational basis
    
    Can do Bell measurements and gradients via shift-rule.

    Input
    circuit_parameters: parameters of circuit
    rotation_gates: Description of circuit rotations
    entangling_gate_index_list: Description of entangling gates used in circuit
    gradient_index: Which parameter index for gradient to compute via shift-rule, set to -1 if no gradient wanted
    sign_shift: Whether to compute A_+ or A_- for gradient
    conjugate: Do complex conjugate for Bell measurement
    get_statevector: Whether we want to use for statevector simulator or Aer simulator
    
    returns circuit
    
    """
    depth=np.shape(circuit_parameters)[0]
    n_qubits=np.shape(circuit_parameters)[1]
    n_qubits_tensor=2*n_qubits
    qreg = qk.QuantumRegister(n_qubits_tensor)

    ##add registers
    if(get_statevector==False):
        creg = qk.ClassicalRegister(n_qubits_tensor)
        circuit = qk.QuantumCircuit(qreg,creg)
    else:
        circuit = qk.QuantumCircuit(qreg)
        
    count_parameters=0
        
        
    ##change sign if conjugate on gates which are complex valued
    if(conjugate==True):
        conjugate_sign=-1
    else:
        conjugate_sign=1
        
        
    ##go through all layers of parameterized quantum circuit
    for j in range(depth):
        ##add parameterized rotation for first copy
        for k in range(n_qubits):
            angle=circuit_parameters[j][k]
            type_pauli=rotation_gates[j][k]
            if(type_pauli==1):
                circuit.rx(conjugate_sign*angle,k)
                ##do gradient shift when we are at parameter to be shifted
                if(count_parameters==gradient_index):
                    circuit.rx(sign_shift*np.pi/2,k)
            elif(type_pauli==2):
                circuit.ry(angle,k)
                ##do gradient shift when we are at parameter to be shifted
                if(count_parameters==gradient_index):
                    circuit.ry(sign_shift*np.pi/2,k)
            elif(type_pauli==3):
                circuit.rz(conjugate_sign*angle,k)
                ##do gradient shift when we are at parameter to be shifted
                if(count_parameters==gradient_index):
                    circuit.rz(sign_shift*np.pi/2,k)
            
            count_parameters+=1 ##counts number of parameters
                
        ##add parameterized rotation for second copy
        for k in range(n_qubits):
            angle=circuit_parameters[j][k]
            type_pauli=rotation_gates[j][k]
            if(type_pauli==1):
                circuit.rx(angle,k+n_qubits)
            elif(type_pauli==2):
                circuit.ry(angle,k+n_qubits)
            elif(type_pauli==3):
                circuit.rz(angle,k+n_qubits)
        
        if(len(entangling_gate_index_list[j])>0):
            ##add entangling cnot gates for first copy
            for gate_indices in entangling_gate_index_list[j]:
                circuit.cnot(gate_indices[0],gate_indices[1])
                
            ##add entangling cnot gates for second copy
            for gate_indices in entangling_gate_index_list[j]:
                circuit.cnot(gate_indices[0]+n_qubits,gate_indices[1]+n_qubits)
        
    ##bell measurement
    for k in range(n_qubits):
        circuit.cnot(k,k+n_qubits)
        circuit.h(k)
    
    ##add measurements
    if(get_statevector==False):
        circuit.barrier()
        circuit.measure(qreg,creg)
        
    return circuit

def get_qiskit_circuit_single(circuit_parameters,rotation_gates,entangling_gate_index_list, get_statevector=True, registers=False,add_measurements=True):
    """
    creates qiskit circuit for measurement on a single copy, i.e. for Pauli measurements
    """
    depth=np.shape(circuit_parameters)[0]
    n_qubits=np.shape(circuit_parameters)[1]
    
    
    qreg = qk.QuantumRegister(n_qubits)
    if(get_statevector==False):
        creg = qk.ClassicalRegister(n_qubits)
        circuit = qk.QuantumCircuit(qreg,creg)
    else:
        circuit = qk.QuantumCircuit(qreg)
        creg=[]

        
    ##go through all layers of parameterized quantum circuit
    for j in range(depth):
        ##add parameterized rotation for first copy
        for k in range(n_qubits):
            angle=circuit_parameters[j][k]
            type_pauli=rotation_gates[j][k]
            if(type_pauli==1):
                circuit.rx(angle,k)
            elif(type_pauli==2):
                circuit.ry(angle,k)
            elif(type_pauli==3):
                circuit.rz(angle,k)
        
        if(len(entangling_gate_index_list[j])>0):
            ##add entangling cnot gates for first copy
            for gate_indices in entangling_gate_index_list[j]:
                circuit.cnot(gate_indices[0],gate_indices[1])
        
    ##add measurements
    if(get_statevector==False and add_measurements==True):
        circuit.barrier()
        circuit.measure(qreg,creg)

    if(registers==False):
        return circuit
    
    else:
        return circuit, qreg, creg


In [108]:
def get_bitstrings(job_qiskit_result,depolarization_factor=0):
    """
    returns Tsallis SE and Sum over pauli expectation values
    Input:
    n: SE index
    job_qiskit_result: Result from qiskit
    depolarization_factor: Global Depolarization error probability per state

    
    Output:
    sampled_bitstrings: Bitstrings that were sampled
    sampled_bitstrings_probability: Probaility of each bitstring
    
    """
    global_depolarization=depolarization_factor*(2-depolarization_factor) #relates depolarization on invidividual copies to global depolarization over two copies. For Bell measurement, this is exact


    #get estimate of sampling probabilities from measured counts
    dict_counts=job_qiskit_result.get_counts(0)
    sampled_bitstrings=[]
    sampled_bitstrings_probability=[]

    #go through all sampled bitstrings
    for key in list(dict_counts.keys()):
        bitstring=[int(key[i]) for i in range(len(key))]
        counts=dict_counts[key]
        #go through all counts and add each bitstring individually correspondong probability 1/n_samples
        for i in range(counts):
            prob=1/n_samples
            sampled_bitstrings_probability.append(prob)
            if(global_depolarization>0):
                rng_depol=rng.random()
                if(rng_depol<global_depolarization):#with chance of global_depolarization, add random bitstring instead of actually measured one
                    random_bitstring=rng.integers(0,2,2*n_qubits)
                    sampled_bitstrings.append(random_bitstring)
                else:
                    sampled_bitstrings.append(bitstring)
            else:##add bitstrings to sampled bitstrings
                sampled_bitstrings.append(bitstring)

    sampled_bitstrings=np.array(sampled_bitstrings)
    sampled_bitstrings_probability=np.array(sampled_bitstrings_probability)

    return sampled_bitstrings,sampled_bitstrings_probability
    


In [109]:
#random generator used
rng = np.random.default_rng(random_seed)




In [110]:
#define parameters for circuit, not used for t-

circuit_parameters=np.zeros([depth,n_qubits])

if(type_parameters==0): ##random angles
    circuit_parameters=rng.random([depth,n_qubits])*2*np.pi

elif(type_parameters==1): ##random clifford angles with n_nonclifford non-clifford parameters introducing T-gates
    circuit_parameters=rng.integers(0,4,[depth,n_qubits])*np.pi/2
    which_params=np.arange(depth*n_qubits)
    rng.shuffle(which_params)
    for i in range(n_nonclifford):
        circuit_parameters[which_params[i]//n_qubits,which_params[i]%n_qubits]=rng.integers(0,4)*np.pi/2+np.pi/4

elif(type_parameters==2): ##n_nonclifford initial magic states and random clifford angles 
    circuit_parameters=rng.integers(0,4,[depth,n_qubits])*np.pi/2
    which_params=np.arange(n_qubits)
    rng.shuffle(which_params)
    #transform initial states (i.e. first layer) into n_nonclifford magic states on random qubits
    for i in range(n_nonclifford):
        circuit_parameters[0,which_params[i]]+=np.pi/4 ##transform into magic state


In [111]:
##define here various types of circuits
if(type_circuit==0): ##A type magic state
    rotation_gates=np.zeros([depth,n_qubits])
    rotation_gates[0,:]=2
    rotation_gates[1,:]=3
    
    circuit_parameters[0,:]=np.pi/2
    circuit_parameters[1,:]=np.pi/4
    
    entangling_gate_index_list=[[] for j in range(depth)]
    
elif(type_circuit==1): ##T-type magic state
    rotation_gates=np.zeros([depth,n_qubits])
    rotation_gates[0,:]=2
    rotation_gates[1,:]=3
    
    circuit_parameters[0,:]=np.arccos(1/np.sqrt(3))
    circuit_parameters[1,:]=np.pi/4
    
    entangling_gate_index_list=[[] for j in range(depth)]

elif(type_circuit==2): ##yz circuit with CNOT in nearest-neighbor chain configuration
    rotation_gates=np.zeros([depth,n_qubits])
    entangling_gate_index_list=[]
    for j in range(depth):
        if(j%2==0):
            rotation_gates[j,:]=2
            entangling_gate_index_list.append([])
        else:
            rotation_gates[j,:]=3
            if(j<depth-1):
                if(n_qubits==2):
                    entangling_gate_index_list.append([[0,1]])
                else:
                    entangling_gate_index_list.append([[2*j,(2*j+1)] for j in range(n_qubits//2)]+[[2*j+1,(2*j+2)] for j in range((n_qubits-1)//2)])
            else:
                entangling_gate_index_list.append([])

elif(type_circuit==3): ##+ state
    rotation_gates=np.zeros([depth,n_qubits])
    rotation_gates[0,:]=2
    
    circuit_parameters[0,:]=np.pi/2

    entangling_gate_index_list=[[] for j in range(depth)]
    

In [112]:
backend = qk.Aer.get_backend('qasm_simulator')
backend_statevector = qk.Aer.get_backend('statevector_simulator')



In [113]:
##prepare circuit, different details depending whether we use statevector simulator or qasm

qiskit_circuit=get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,get_statevector=False)
qiskit_circuit_single=get_qiskit_circuit_single(circuit_parameters,rotation_gates,entangling_gate_index_list,get_statevector=True)
if(gradient_index>=0):
    shifted_circuitPlus=get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,gradient_index=gradient_index,sign_shift=1,get_statevector=False)
    shifted_circuitMinus=get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,gradient_index=gradient_index,sign_shift=-1,get_statevector=False)


In [114]:
##run job
job_qiskit = qk.execute(qiskit_circuit,backend=backend,shots=n_samples)
job_qiskit_single = qk.execute(qiskit_circuit_single,backend=backend_statevector)

if(gradient_index>=0):
    job_shiftedPlus = qk.execute(shifted_circuitPlus,backend=backend,shots=n_samples)
    job_shiftedMinus = qk.execute(shifted_circuitMinus,backend=backend,shots=n_samples)
    


In [115]:
##get result from IBMQ
job_qiskit_result=job_qiskit.result()
job_qiskit_result_single=job_qiskit_single.result()
if(gradient_index>=0):
    job_shiftedPlus_result = job_shiftedPlus.result()
    job_shiftedMinus_result = job_shiftedMinus.result()


In [116]:
##evaluate counts by turning them into a probability vector
##note that we need to transform from little to big endian for the probability vector

print("Get SE efficiently for even n (for odd index scales exponentially with qubit number)")
print("Using",n_samples,"shots")

#get state and exact SE
psi = job_qiskit_result_single.get_statevector()
print(psi)
SE_exact = se_exact(n, n_qubits, psi, backend)


##calculate 
sampled_bitstrings,sampled_bitstrings_probability = get_bitstrings(job_qiskit_result,depolarization_factor)



SE=se_sample(n, sampled_bitstrings,sampled_bitstrings_probability, n_samples=n_samples, n_resample=n_resample)




print("Tsallis SE (sampled)",SE)
print("Tsallis SE (exact)",SE_exact)





Get SE efficiently for even n (for odd index scales exponentially with qubit number)
Statevector([ 0.13578079+0.13322294j, -0.49235582-0.15769479j,
              0.11176053-0.51609682j,  0.04164138+0.17847936j,
             -0.39356946+0.11971911j, -0.05513498+0.15007506j,
             -0.29290225+0.01490487j, -0.13823191-0.2901451j ],
            dims=(2, 2, 2))
Tsallis SE (sampled) 0.35626
Tsallis SE (exact) 0.35831771128467904


In [117]:
#mitigate Tsallis SE

print("Mitigate Tsallis SE against depolarizing error")

##calculate purity
purity=SWAP_purity(sampled_bitstrings,sampled_bitstrings_probability)
print("Purity",purity)

hilbertspace=2**n_qubits
prob_Identity_addedstring=np.sum(sampled_bitstrings_probability**2) #probability of same bitstrings appearing, lower bounded by 4**-n_qubits and upper bounded by 2**-n_qubits

depol_probability = 1-np.sqrt((hilbertspace-1)*(hilbertspace*purity-1))/(hilbertspace-1) ##get depolariziation probability from purity

factor = (1-depol_probability)**(2*n)
SE_mtg = (SE - (1-factor)*(2**-n_qubits -1)/(1-n))/factor

print("Measured depolarization probability",depol_probability)
print("Mitigated Tsallis SE (sampled)", SE_mtg)


Mitigate Tsallis SE against depolarizing error
Purity 1.0
Measured depolarization probability 0.0
Mitigated Tsallis SE (sampled) 0.35626


In [118]:
print("Running efficient method for any index n by using complex conjugate of states using ",n_samples_cc,"shots")
se_conjugate = se_sample_conjugate(n, circuit_parameters, rotation_gates, entangling_gate_index_list, n_samples_cc)
print("Tsallis SE (exact)",SE_exact)
print("Tsallis SE sampled (using cc)",se_conjugate)

Running efficient method for any index n by using complex conjugate of states
Tsallis SE (exact) 0.35831771128467904
Tsallis SE sampled (using cc) 0.373


In [121]:
##calculating the gradient of SE via sampling and shift-rule


if(gradient_index>=0):
    print("Estimate gradient with shift-rule using",n_samples,"shots")
    ##calculate plus shifted
    sampled_bitstrings_plus,sampled_bitstrings_probability_plus = get_bitstrings(job_shiftedPlus_result)

    ##calculate minus shifted
    sampled_bitstrings_minus,sampled_bitstrings_probability_minus = get_bitstrings(job_shiftedMinus_result)

    ##A corresponds to sum over pauli expectation values^2n
    A_plus=A_sample_gradient(n, sampled_bitstrings,sampled_bitstrings_probability, sampled_bitstrings_plus,
                      sampled_bitstrings_probability_plus, n_samples=n_samples, n_resample=n_resample)

    A_minus=A_sample_gradient(n, sampled_bitstrings,sampled_bitstrings_probability, sampled_bitstrings_minus,
                      sampled_bitstrings_probability_minus, n_samples=n_samples, n_resample=n_resample)

    ##prefactor n due to adapted shift-rule for 2n copies
    
    gradient_sampled=n*(A_plus-A_minus)/(1-n)

    print("Tsallis SE gradient sampled",gradient_sampled , "for circuit parameter index",gradient_index)

Estimate gradient with shift-rule using 10000 shots
Tsallis SE gradient sampled 0.034110000000000015 for circuit parameter index 0


In [122]:
##compute exact gradient by finite difference method
if(gradient_index>=0):
    epsilon=10**-10
    circuit_parameters_plus=np.array(circuit_parameters)
    circuit_parameters_plus[gradient_index//n_qubits,gradient_index%n_qubits]+=epsilon

    qiskit_circuit_single=get_qiskit_circuit_single(circuit_parameters_plus,rotation_gates,entangling_gate_index_list,get_statevector=True)
    job_qiskit_single = qk.execute(qiskit_circuit_single,backend=backend_statevector)
    job_qiskit_result_single=job_qiskit_single.result()
    psi = job_qiskit_result_single.get_statevector()
    SE_exact_plus = se_exact(n, n_qubits, psi, backend)


    SE_exact_gradient=(SE_exact_plus-SE_exact)/epsilon

    print("Tsallis SE exact gradient", SE_exact_gradient, "for circuit parameter index",gradient_index)

Tsallis SE exact gradient 0.031025737534662312 for circuit parameter index 0
